<a href="https://colab.research.google.com/github/SIDIBEMoussa/Simulation-ev-n-ment-discret/blob/main/M_M_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import matplotlib.pyplot as pl
import pandas as pd

In [ ]:
Clients=np.array(range(1,1001))
Uniforme1=np.array(np.random.rand(1000))
Uniforme2=np.array(np.random.rand(1000))

In [ ]:
def condition(U):
  T=[]
  for e in U:
    if e<0.1:
      T.append(1)
    elif e<0.2:
      T.append(2)
    elif e<0.3:
      T.append(3)
    elif e<0.4:
      T.append(4)
    elif e<0.5:
      T.append(5)
    elif e<0.6:
      T.append(6)
    elif e<0.7:
      T.append(7)
    elif e<0.8:
      T.append(8)
    elif e<0.9:
      T.append(9)
    else:
      T.append(10)
  return np.array(T)

def condition2(U):
  T=[]
  for e in U:
    if e<0.25:
      T.append(2)
    elif e<0.5:
      T.append(3)
    elif e<0.75:
      T.append(4)
    else:
      T.append(5)
  return np.array(T)

In [ ]:
T_inter_arr=condition(Uniforme1)

In [ ]:
T_arr=[T_inter_arr[0]]
for i in range(1,len(Clients)):
  T_arr.append(T_inter_arr[i]+T_arr[i-1])
T_arr=np.array(T_arr)

In [ ]:
T_service=condition2(Uniforme2)

In [ ]:
debut_ser=[T_arr[0]]
fin_ser=[debut_ser[0]+T_service[0]]

for i in range(1, len(Clients)):
    debut_ser.append(max(T_arr[i],fin_ser[i-1]))
    fin_ser.append(T_service[i]+debut_ser[i])

debut_ser=np.array(debut_ser)
fin_ser=np.array(fin_ser)

In [ ]:
T_attente=debut_ser-T_arr

In [ ]:
T_in_syst=fin_ser-T_arr

In [ ]:
server_free_time=[np.NaN]
for i in range(1,len(Clients)):
  server_free_time.append(max(0,T_arr[i]-fin_ser[i-1]))
server_free_time=np.array(server_free_time)

In [ ]:
M_M_1=pd.DataFrame({"Client":Clients,"U[0,1]":Uniforme1,"T_inter_arr":T_inter_arr,"T_arrivé":T_arr,"debut_ser":debut_ser,"U[0,1]":Uniforme2,"T_service":T_service,"fin_service":fin_ser,"T_attente":T_attente,"T_ds_systeme":T_in_syst,"T_Server_free":server_free_time})

In [ ]:
M_M_1

,Client,"U[0,1]",T_inter_arr,T_arrivé,debut_ser,T_service,fin_service,T_attente,T_ds_systeme,T_Server_free
0,1,0.287240,10,10,10,3,13,0,3,NaN
1,2,0.136972,6,16,16,2,18,0,2,3.0
2,3,0.297357,9,25,25,3,28,0,3,7.0
3,4,0.122499,8,33,33,2,35,0,2,5.0
4,5,0.255111,7,40,40,3,43,0,3,5.0
...,...,...,...,...,...,...,...,...,...,...
995,996,0.512895,4,5340,5340,4,5344,0,4,1.0
996,997,0.023932,7,5347,5347,2,5349,0,2,3.0
997,998,0.090963,7,5354,5354,2,5356,0,2,5.0
998,999,0.141223,8,5362,5362,2,5364,0,2,6.0


In [ ]:
mu=1/np.mean(M_M_1.T_service)
lamda=1/np.mean(M_M_1.T_inter_arr)
ro=lamda/mu
mu,lamda,ro

(0.2839295854628052, 0.18628912071535023, 0.6561102831594635)

In [ ]:
def GenerateMM(Nbr_cl):
  np.random.seed(100)
  t_inter_arr=int(np.random.uniform(0,1)*10)
  t_debut_ser=t_arr=t_inter_arr
  t_ser=int(np.random.uniform(0,0.51)*10)
  t_fin_ser=t_inter_arr+t_ser
  t_sejour=t_fin_ser-t_arr
  t_ser_libre=0
  t_attente=t_debut_ser-t_arr
  #Cumul
  
  sejour_cumul=t_sejour
  arr_cumul=t_arr
  servi_cumul=t_ser
  inter_arr_cumul=t_inter_arr
  sejour_cumul=t_attente+t_ser
  t_ser_libre_cumul=0
  n=1

  #check=[t_inter_arr]
  while n<=Nbr_cl:
      t_inter_arr=int(np.random.uniform(0,1.01)*10)
      t_arr+=t_inter_arr
      t_ser_libre=max(0,t_arr-t_fin_ser)
      t_debut_ser=max(t_fin_ser,t_arr)
      t_attente=t_debut_ser-t_arr
      t_ser=int(np.random.uniform(0,0.6)*10)
      t_fin_ser=t_debut_ser+t_ser
      t_sejour=t_attente+t_ser
      
      #check.append(t_inter_arr)
      
      #Cumul
      
      #att_cumul+=t_attente
      arr_cumul+=t_arr
      servi_cumul+=t_ser
      inter_arr_cumul+=t_inter_arr
      sejour_cumul+=t_sejour
      t_ser_libre_cumul+=t_ser_libre
      n=n+1
  df=pd.DataFrame({"Nbre_client":[n-1],"arrivé_cum":[arr_cumul],"service_cum":[servi_cumul],"inter_arrivé_cum":[inter_arr_cumul],"sejour_cum":[sejour_cumul],"server_libre_cumul":[t_ser_libre_cumul]})
  return df, check


In [ ]:
df,check=GenerateMM(1000)
df

,Nbre_client,arrivé_cum,service_cum,inter_arrivé_cum,sejour_cum,server_libre_cumul
0,1000,2238237,2447,4401,3657,1955


In [ ]:
lamda1=float(1/(df.inter_arrivé_cum/df.Nbre_client))
mu1=float(1/(df.service_cum/df.Nbre_client))
ro=lamda1/mu1
(mu1,lamda1,ro)

(0.40866366979975477, 0.22722108611679165, 0.5560099977277893)

In [ ]:
nbr_cl_syst=ro/(1-ro)
nbr_cl_file=nbr_cl_syst-ro
duree_sejour=nbr_cl_syst/lamda1

In [ ]:
print("le taux d'occupation du server est:{:.2%}".format(ro))
print("le nombre moyen de client dans le système:{:.2} personne(s)".format(nbr_cl_syst))
print("le nombre moyen de client dans la file d'attente:{:.2} personne(s)".format(nbr_cl_file))
print("La durée moyenne de séjour:{:.2} minutes".format(duree_sejour))

le taux d'occupation du server est:55.60%
le nombre moyen de client dans le système:1.3 personne(s)
le nombre moyen de client dans la file d'attente:0.7 personne(s)
La durée moyenne de séjour:5.5 minutes
